In [1]:
import pickle

import pandas as pd
from sklearn.svm import SVC
from xgboost import XGBClassifier

from module.mymodule import best_parameters, retrained
from features import pipe_1, pipe_2, pipe_3, pipe_4, pipe_5, pipe_6, pipe_7,\
                     pipe_8, pipe_9, pipe_10, pipe_11, pipe_12, pipe_13, pipe_14

### オリジナルデータの呼び出し

In [2]:
df = pd.read_csv('./data/train.csv')

### 

### パイプライン候補からデータセットの作成

In [3]:
df = pd.read_csv('./data/train.csv')
to_pipe ={
            'df': df,
            'split_kwrg': {},
            'train_flg': True,
            'retrain': True,
            }

pipe_lines = [
            pipe_1,  # base 
            pipe_2,  # StSlopeCat
            pipe_3,  # CholsetMean
            pipe_4,  # AgeCat
            pipe_5,  # StSlopeCat CholetMean AgeCat
            pipe_6,  # RestingBpCat
            pipe_7,  # OldPeakCat
            pipe_8,  # RestingBpCat OldPeakCat
            pipe_9,  # Onehot
            pipe_10,  # CholestMean AgeCat Onehot
            pipe_11,  # CholCut
            pipe_12,  # CholCut Onehot,
            pipe_13,  # DropByShap
            pipe_14,
            ]  # 特徴量候補を設定
data_set = {pipe.__name__: pipe(**to_pipe) for pipe in pipe_lines}

                                Base(pipe_1)                               


,Age,Sex,RestingBP,Cholesterol,FastingBS,MaxHR,ExerciseAngina,Oldpeak
0,56,1,155,342,1,150,1,3.0
1,55,0,130,394,0,150,0,0.0
2,47,1,110,0,1,120,1,0.0


                        StSlpoeCat Standard(pipe_2)                        


,Age,Sex,RestingBP,Cholesterol,FastingBS,MaxHR,ExerciseAngina,Oldpeak
0,56,1,155,342,1,150,1,3.0
1,55,0,130,394,0,150,0,0.0
2,47,1,110,0,1,120,1,0.0


                        CholestMean Standard(pipe_3)                       


...

,Age,Sex,RestingBP,Cholesterol,FastingBS,MaxHR,ExerciseAngina,Oldpeak,ChestPainType_ASY,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA,RestingECG_LVH,RestingECG_Normal,RestingECG_ST,ST_Slope_Down,ST_Slope_Flat,ST_Slope_Up
0,56,1,155,342,1,150,1,3.0,1,0,0,0,0,1,0,0,1,0
1,55,0,130,394,0,150,0,0.0,0,1,0,0,1,0,0,0,0,1
2,54,0,160,201,0,163,0,0.0,0,0,1,0,0,1,0,0,0,1


                            DropByShap(pipe_13)                            


,Age,Sex,Cholesterol,FastingBS,MaxHR,Oldpeak,ChestPainType_ASY,ST_Slope_Flat,ST_Slope_Up
0,56,1,342,1,150,3.0,1,1,0
1,55,0,394,0,150,0.0,0,0,1
2,47,1,0,1,120,0.0,0,1,0


                        ZeroCat DropByShap(pipe_14)                        


,Age,Sex,Cholesterol,FastingBS,MaxHR,Oldpeak,ChestPainType_ASY,ST_Slope_Flat,ST_Slope_Up
0,56,1,342,1,150,3.0,1,1,0
1,55,0,394,0,150,0.0,0,0,1
2,54,0,201,0,163,0.0,0,0,1


### グリットサーチで訓練したモデルの呼び出し

In [4]:
file_name = 'mixed'
with open(f'./data/{file_name}.pkl', 'rb') as f:
    trained_models = pickle.load(f)

### 最適なパラメータで再訓練

In [6]:
retrain = [
            XGBClassifier,
            SVC
            ]

best_param = best_parameters(trained_models, pipe_lines)
# 再訓練してpickleで保存
retrained_models = retrained(retrain, pipe_lines, data_set, best_param, file_name)